# HDF5 to VDF

## Converting the File

In [1]:
# import the generic munger
from util.HDF5_vdf_convert import GenericVDFConverter

In [2]:
#Import necessary packages for HDF5
import h5py
import numpy as np
import pandas as pd
import os
from batdata.data import BatteryDataset
import yaml
import pprint

In [5]:
#For single file conversion
file_path='sample_files/example_HDF5.h5'
config_path='example_HDF5_config.yaml'
dataset=BatteryDataset.from_batdata_hdf(file_path)
#BatteryDataset.get_metadata_from_hdf5(file_path)
title=file_path.replace('.h5','.csv')
dataset.raw_data.to_csv(title, sep='\t')
converter = GenericVDFConverter(title, config_path)
converter.convert()
if(os.path.exists(title) and os.path.isfile(title)):
    os.remove(title)
    print("file deleted")
else:
    print("file not found")

Reading in data...
Formatting data...
Updating header dictionary...
Adding units...


/Users/evasabine/opt/anaconda3/lib/python3.9/site-packages/pandas/core/base.py:1055: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(self).is_monotonic


Completed generation of VDF file at path: sample_files/VDF/example_HDF5_VDF.csv
file deleted


In [7]:
#For multiple file conversion with File Titles and Timestamps in csv columns
name_and_time = pd.read_csv('example_HDF5_times.csv')
#accepts input of unix timestamp
name_and_time['File Name'].astype('string')
name_and_time['Datetime']=pd.to_datetime(name_and_time['Timestamp'], unit='ms')
array=name_and_time['Datetime']
for i in range(0,299):
    file_path='sample_files/'+name_and_time['File Name'].iloc[i]
    dataset=BatteryDataset.from_batdata_hdf(file_path)
    title=file_path.replace('.h5','.csv')
    dataset.raw_data.to_csv(title, sep='\t')
    with open("example_HDF5_config.yaml") as f:
        config=yaml.unsafe_load(f)
    #pprint.pprint(config)
    #converts timestamp to ISO 8601
    starttime=array[i].to_pydatetime().isoformat()
    config['metadata']['Start Time']=starttime
    pprint.pprint(config)
    with open("example_HDF5_config.yaml", "w") as f:
        yaml.dump(config, f)
    config_path = (
        "example_HDF5_config.yaml"
    )
    file_path = (
        title
    )
    converter = GenericVDFConverter(file_path, config_path)
    converter.read_data()
    converter.convert()
    if(os.path.exists(title) and os.path.isfile(title)):
        os.remove(title)
end

{'columns': {'current': {'rename': 'Current', 'unit': 'amp'},
             'cycle_number': {'rename': 'Cycle Number'},
             'method': {'rename': 'Method'},
             'state': {'rename': 'State'},
             'step_index': {'rename': 'Step Index'},
             'substep_index': {'rename': 'Substep Index'},
             'test_time': {'rename': 'Test Time', 'unit': 'second'},
             'voltage': {'rename': 'Potential', 'unit': 'volt'}},
 'metadata': {'Start Time': '2012-03-28T18:40:00',
              'Timezone': 'America/Chicago'}}
Reading in data...
Formatting data...
Updating header dictionary...
Adding units...


/Users/evasabine/opt/anaconda3/lib/python3.9/site-packages/pandas/core/base.py:1055: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(self).is_monotonic


Completed generation of VDF file at path: sample_files/VDF/example_HDF5_VDF.csv
{'columns': {'current': {'rename': 'Current', 'unit': 'amp'},
             'cycle_number': {'rename': 'Cycle Number'},
             'method': {'rename': 'Method'},
             'state': {'rename': 'State'},
             'step_index': {'rename': 'Step Index'},
             'substep_index': {'rename': 'Substep Index'},
             'test_time': {'rename': 'Test Time', 'unit': 'second'},
             'voltage': {'rename': 'Potential', 'unit': 'volt'}},
 'metadata': {'Start Time': '2012-01-31T17:13:20',
              'Timezone': 'America/Chicago'}}
Reading in data...
Formatting data...
Updating header dictionary...
Adding units...


/Users/evasabine/opt/anaconda3/lib/python3.9/site-packages/pandas/core/base.py:1055: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(self).is_monotonic


Completed generation of VDF file at path: sample_files/VDF/example1_HDF5_VDF.csv


TypeError: can only concatenate str (not "float") to str

In [ ]:
#if input timestamp information is in datetime format, substitute following code
starttime=name_and_time['Timestamp'].to_pydatetime().isoformat()


## Inspecting the Results

In [ ]:
# Showing the metadata that was added to the VDF file.
munger.header_dict

In [ ]:
# Take a look at the VDF time data (units included)
munger.final_df.head()

In [ ]:
# Take a look at the raw time data read in from the original file
munger.time_df.head()

In [ ]:
# Take a look at the formatted data (without the added units)
munger.formatted_df.head()

In [ ]:
# Visualize the data for a specific column
munger.formatted_df.plot(x="Test Time", y="Current")